In [269]:
from tqdm import tqdm
import pymssql
import pandas as pd
import numpy as np
import warnings
import glob 
import os

warnings.filterwarnings('ignore')

In [125]:
conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [Код склада], [Код товара], [Артикул поставщика], [Внутренний артикул], [ID Сайта], [Товарное направление],
[Товарная группа], [Товарный кластер], [Проба], [Размер изделия], [Тип изделия 1], [Тип изделия 2],
[Тип изделия 3], [Дизайн], [Бренд], [Гендерный признак], [Ценовая корзина], [Группа цен], [Остаток], [Цена Розн., за шт],
[Чистый вес], [Цена изделия на бирке], [Скидка на изделии], [Запрет скидки], [Цена Закупки за 1 шт. (упр сс с НДС за единицу)], [Дата закупки]
from [Staging].[Reports].[Remainings_8h]
where [Товарное направление] in (N'БК', N'ЦБ', N'ИФ', N'ПДК', N'ДК')
'''

sql_query = pd.read_sql_query(sql, conn)
stock = pd.DataFrame(sql_query)

In [126]:
images = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Артикул-фото 2024.06.11_.xlsx')

In [127]:
zapret = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\запрет Артикулы к размещению.xlsx')

In [128]:
kniga = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Книга11.xlsx')

In [129]:
os.chdir(r'C:\\Users\Trenkin.Sergey\Desktop\sales')
extension = 'xlsx'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
all_filenames

['2024-03-10_2024-03-19.xlsx',
 '2024-03-20_2024-03-31.xlsx',
 '2024-04-01_2024-04-09.xlsx',
 '2024-04-10_2024-04-19.xlsx',
 '2024-04-20_2024-04-30.xlsx',
 '2024-05-01_2024-05-09.xlsx',
 '2024-05-10_2024-05-19.xlsx',
 '2024-05-20_2024-05-31.xlsx',
 '2024-06-01_2024-06-09.xlsx',
 '2024-06-10_2024-06-10.xlsx',
 '2024-06-10_2024-06-12.xlsx',
 '2024-06-10_2024-06-18.xlsx']

In [130]:
sale = pd.DataFrame()
for name in tqdm(all_filenames):
    sales = pd.read_excel(name)
    sale = pd.concat([sales, sale], ignore_index= True)

100%|██████████| 12/12 [12:21<00:00, 61.81s/it]


In [131]:
sale['Дата'] = pd.to_datetime(sale['Дата'], dayfirst=True, format='mixed')

In [132]:
sale = sale[(sale['Дата'] >= pd.to_datetime('2024-03-14', dayfirst=True, format='mixed')) & 
            (sale['Дата'] <= pd.to_datetime('2024-06-19', dayfirst=True, format='mixed'))]

In [133]:
sale = sale.groupby(by = 'Тов', as_index=False)[['Количество', 'Общая сумма']].sum()

In [134]:
ids = stock[['Код товара', 'ID Сайта']]
ids.columns = ['Тов', 'ID Сайта']

In [135]:
sale = sale.merge(ids, how='left', on='Тов')
sale = sale.drop(columns=['Тов'])
sale = sale[['ID Сайта', 'Количество', 'Общая сумма']]

In [136]:
kniga['Код склада'] = kniga['Код склада'].astype(str)
skladi = kniga['Код склада'].unique().tolist()

In [137]:
stock.columns

Index(['Код склада', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
       'ID Сайта', 'Товарное направление', 'Товарная группа',
       'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
       'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
       'Гендерный признак', 'Ценовая корзина', 'Группа цен', 'Остаток',
       'Цена Розн., за шт', 'Чистый вес', 'Цена изделия на бирке',
       'Скидка на изделии', 'Запрет скидки',
       'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Дата закупки'],
      dtype='object')

In [138]:
stock.columns = ['code', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
    'ID Сайта', 'Товарное направление', 'Товарная группа',
    'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
    'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
    'Гендерный признак', 'Ценовая корзина', 'Группа цен', 'Остаток', 'Цена Розн., за шт',
    'Чистый вес', 'Цена изделия на бирке', 'Скидка на изделии',
    'Запрет скидки', 'Цена Закупки за 1 шт. (упр сс с НДС за единицу)',
    'Дата закупки']

In [139]:
stock = stock.query('code in @skladi')

In [140]:
stock.columns = ['Код склада', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
    'ID Сайта', 'Товарное направление', 'Товарная группа',
    'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
    'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
    'Гендерный признак', 'Ценовая корзина', 'Группа цен', 'Остаток', 'Цена Розн., за шт',
    'Чистый вес', 'Цена изделия на бирке', 'Скидка на изделии',
    'Запрет скидки', 'Цена Закупки за 1 шт. (упр сс с НДС за единицу)',
    'Дата закупки']

In [141]:
result = stock.drop_duplicates(subset=['ID Сайта'])

In [142]:
tovs = result['ID Сайта'].dropna().tolist()

In [143]:
images['article'] = images['article'].astype(str)

In [144]:
images = images.query('article in @tovs')

In [145]:
images = images[['article', 'published', 'Ссылка на изображение']]

In [146]:
images.columns = ['ID Сайта', 'Опубликован', 'Ссылка на изображение']

In [147]:
result = result.merge(images, how='outer', on='ID Сайта')

In [148]:
result = result.drop(columns = ['Остаток'])

In [149]:
df = stock

In [150]:
stock = df.groupby(by = 'ID Сайта', as_index=False)['Остаток'].sum()

In [151]:
result = result.merge(stock, how='left', on='ID Сайта')

In [152]:
sale.columns = ['ID Сайта', 'Количество продаж 14.03-19.06', 'Сумма продаж 14.03-19.06']

In [153]:
result = result.merge(sale, how='left', on='ID Сайта')

In [154]:
result.columns

Index(['Код склада', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
       'ID Сайта', 'Товарное направление', 'Товарная группа',
       'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
       'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
       'Гендерный признак', 'Ценовая корзина', 'Группа цен',
       'Цена Розн., за шт', 'Чистый вес', 'Цена изделия на бирке',
       'Скидка на изделии', 'Запрет скидки',
       'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Дата закупки',
       'Опубликован', 'Ссылка на изображение', 'Остаток',
       'Количество продаж 14.03-19.06', 'Сумма продаж 14.03-19.06'],
      dtype='object')

In [155]:
result = result.drop_duplicates(subset=['ID Сайта'])

In [156]:
df['Чистый вес'] = df['Чистый вес'].astype(float)

In [157]:
clear_weight = df.groupby(by = 'ID Сайта', as_index=False)['Чистый вес'].agg({'min', 'median', 'max'})
clear_weight.columns = ['ID Сайта', 'Чистый вес макс', 'Чистый вес медиана', 'Чистый вес мин']
clear_weight['Чистый вес размах, %'] = (clear_weight['Чистый вес макс'] / clear_weight['Чистый вес мин'] - 1) *-1

In [158]:
clear_weight

,ID Сайта,Чистый вес макс,Чистый вес медиана,Чистый вес мин,"Чистый вес размах, %"
0,,0.206,21.280,1.974,0.895643
1,1000054,0.980,1.150,1.080,0.092593
2,1000091,0.944,1.154,1.059,0.108593
3,1000339,2.092,2.252,2.162,0.032377
4,1000450,3.540,3.540,3.540,-0.000000
...,...,...,...,...,...
44603,8380055,2.050,2.140,2.090,0.019139
44604,8380061,1.670,1.708,1.700,0.017647
44605,8380063,1.510,1.930,1.770,0.146893
44606,8380071,2.370,2.610,2.520,0.059524


In [159]:
result = result.merge(clear_weight, how='left', on='ID Сайта')

In [160]:
price_birka = df.groupby(by = 'ID Сайта', as_index=False)['Цена изделия на бирке'].agg({'min', 'median', 'max'})
price_birka.columns = ['ID Сайта', 'Цена изделия на бирке макс', 'Цена изделия на бирке медиана', 'Цена изделия на бирке мин']

In [161]:
price_birka

,ID Сайта,Цена изделия на бирке макс,Цена изделия на бирке медиана,Цена изделия на бирке мин
0,,510.0,553886.0,79836.0
1,1000054,21972.0,30225.0,22499.0
2,1000091,9800.0,37888.0,37888.0
3,1000339,87065.0,104042.0,88906.0
4,1000450,873430.0,873430.0,873430.0
...,...,...,...,...
44603,8380055,33986.0,48832.0,47256.0
44604,8380061,31071.0,40623.0,37566.0
44605,8380063,25516.0,71439.0,38843.0
44606,8380071,37087.0,61569.0,40029.0


In [162]:
result = result.merge(price_birka, how='left', on='ID Сайта')

In [163]:
base_discount = df.groupby(by = 'ID Сайта', as_index=False)['Скидка на изделии'].agg({'min', 'max'})
base_discount.columns = ['ID Сайта', 'Базовая скидка макс', 'Базовая скидка мин']

In [164]:
base_discount

,ID Сайта,Базовая скидка макс,Базовая скидка мин
0,,0.0,71.0
1,1000054,53.0,57.0
2,1000091,0.0,66.0
3,1000339,62.0,67.0
4,1000450,0.0,0.0
...,...,...,...
44603,8380055,37.0,45.0
44604,8380061,37.0,38.0
44605,8380063,32.0,66.0
44606,8380071,24.0,59.0


In [165]:
result = result.merge(base_discount, how='left', on='ID Сайта')

In [166]:
df['Цена со скидками'] = np.where(
    df['Запрет скидки'] == 'Да', df['Цена Розн., за шт'], df['Цена Розн., за шт'] * (1-df['Скидка на изделии']/100) * 0.8 * 0.6
)

In [167]:
price_skidki = df.groupby(by = 'ID Сайта', as_index=False)['Цена со скидками'].agg({'min', 'median', 'max'})
price_skidki.columns = ['ID Сайта', 'Цена со скидками макс', 'Цена со скидками медиана', 'Цена со скидками мин']
price_skidki['Цена со скидками размах, %'] = (price_skidki['Цена со скидками макс'] / price_skidki['Цена со скидками мин'] - 1) * -1

In [168]:
result = result.merge(price_skidki, how='left', on='ID Сайта')

In [169]:
selfprice = df.groupby(by = 'ID Сайта', as_index=False)['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'].agg({'min', 'median', 'max'})
selfprice.columns = ['ID Сайта', 'Себестоимость макс', 'Себестоимость медиана', 'Себестоимость мин']
selfprice['Себестоимость размах, %'] = (selfprice['Себестоимость макс'] / selfprice['Себестоимость мин'] - 1) * -1

In [170]:
result = result.merge(selfprice, how='left', on='ID Сайта')

In [171]:
df['Наценка'] = df['Цена со скидками'] / df['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'] - 1

In [172]:
markup = df.groupby(by = 'ID Сайта', as_index=False)['Наценка'].agg({'min', 'median', 'max'})
markup.columns = ['ID Сайта', 'Наценка макс', 'Наценка медиана', 'Наценка мин']

In [173]:
result = result.merge(markup, how='left', on='ID Сайта')

In [174]:
zapret['Артикул сайта'] = zapret['Артикул сайта'].astype(str)

In [175]:
zapret = zapret['Артикул сайта'].tolist()

In [176]:
result['Запрет к размещению'] = np.nan
for i in result.index:
    if result['ID Сайта'][i] in zapret:
        result['Запрет к размещению'][i] = True
    else:
        result['Запрет к размещению'][i] = False

In [177]:
ecom = pd.read_excel(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\ecom_stock_raw\Остатки 2024.06.19.xlsx')

In [178]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\discounts_zolotoy\\'
list_of_files = glob.glob(path + '\*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

discount_ecom = pd.read_excel(latest_file)

In [179]:
discount_ecom.columns = ['article', 'Опубликован', 'discount_value_promotions_1', 'discount_value_promotions_2', 'discount_value_promotions_3', 'discount_value_promotions_4']

In [180]:
discount_ecom = discount_ecom.drop(columns=['discount_value_promotions_2', 'discount_value_promotions_3', 'discount_value_promotions_4'])

In [181]:
ecom = ecom.merge(discount_ecom, how='left', on='article')

In [182]:
def discount_index(discount):
    try:
        return 1 - discount / 100
    except:
        return 1

In [183]:
ecom.discount_value_promotions_1 = ecom.discount_value_promotions_1.apply(discount_index)

In [184]:
ecom['Цена со скидками Ecom'] = np.where(
    (ecom['price'] == ecom['discount_price']) & (ecom['discount_value_promotions_1'] == 1), ecom['discount_price'] * ecom['discount_value_promotions_1'], 
    np.where(
        ecom['discount_value_promotions_1'] == 0.5, ecom['discount_price'] * ecom['discount_value_promotions_1'], ecom['discount_price'] * ecom['discount_value_promotions_1'] * 0.8
    )
)

In [185]:
ecom['article'] = ecom['article'].astype(str)
ecom = ecom[['article', 'size', 'Цена со скидками Ecom']]

In [186]:
ecom = ecom.query('article in @tovs')

In [187]:
ecom.columns = ['ID Сайта', 'size', 'Цена со скидками Ecom']

In [188]:
result = result[result['ID Сайта'] != '']

In [189]:
result['ID Сайта'] = result['ID Сайта'].astype(int).astype(str)
result['Размер изделия'] = result['Размер изделия'].astype(str)

In [190]:
ecom['size'] = ecom['size'].fillna(0.0)
ecom['size'] = ecom['size'].astype(str)

In [191]:
ecom['merge'] = ecom['ID Сайта'] + ecom['size']

In [192]:
result['merge'] = result['ID Сайта'] + result['Размер изделия']

In [193]:
ecom = ecom.drop(columns=['ID Сайта', 'size'])

In [194]:
result = result.merge(ecom, how='outer', on='merge')

In [195]:
result = result.drop(columns=['merge'])

In [196]:
result = result.dropna(subset=['ID Сайта'])

In [197]:
df['Сумма в себестоимости на остатках'] = df['Остаток'] * df['Цена Закупки за 1 шт. (упр сс с НДС за единицу)']

In [198]:
summa_ost = df.groupby(by ='ID Сайта', as_index=False)['Сумма в себестоимости на остатках'].sum()

In [199]:
result = result.merge(summa_ost, how='left', on='ID Сайта')

In [200]:
df['Цена за грамм'] = df['Цена со скидками'] / df['Чистый вес']

In [201]:
price_per_gramm = df.groupby(by = 'ID Сайта', as_index=False)['Цена за грамм'].agg({'min', 'median', 'max'})
price_per_gramm.columns = ['ID Сайта', 'Цена за грамм макс', 'Цена за грамм медиана', 'Цена за грамм мин']

In [202]:
result['Себестоимость размах, %'].value_counts()

Себестоимость размах, %
-0.000000    12052
 1.000000        5
 0.582508        4
 0.015385        3
 0.047619        3
             ...  
 0.135848        1
 0.322675        1
 0.033544        1
 0.072259        1
 0.157813        1
Name: count, Length: 32522, dtype: int64

In [203]:
result = result.merge(price_per_gramm, how='left', on='ID Сайта')

In [204]:
result.to_csv('result.csv', index=False)

In [205]:
zapret_sk = df.groupby(by = 'ID Сайта', as_index=False)['Запрет скидки'].count()

In [206]:
yes = df[df['Запрет скидки'] == 'Да']
no = df[df['Запрет скидки'] == 'Нет']

In [207]:
yes = yes.groupby(by = 'ID Сайта', as_index=False)['Запрет скидки'].count()
no = no.groupby(by = 'ID Сайта', as_index=False)['Запрет скидки'].count()

In [208]:
yes.columns = ['ID Сайта', 'Количество с запрещенной скидкой']

In [209]:
no.columns = ['ID Сайта', 'Количество с разрешенной скидкой']

In [210]:
yes = yes.merge(no, how='outer', on='ID Сайта')

In [211]:
result = result.merge(yes, how='left', on='ID Сайта')

In [212]:
result = result.drop_duplicates(subset=['ID Сайта'])

In [213]:
df['Дата закупки'] = pd.to_datetime(df['Дата закупки'], dayfirst=True, format='mixed')

In [214]:
df['Старая закупка (до 31.12.2022)'] = np.where(
    df['Дата закупки'] <= pd.to_datetime('2022-12-31', dayfirst=True, format='mixed'), 1, 0
)

In [215]:
df['Новая закупка (с 01.01.2023)'] = np.where(
    df['Дата закупки'] >= pd.to_datetime('2023-01-01', dayfirst=True, format='mixed'), 1, 0
)

In [216]:
data = df.groupby(by = 'ID Сайта', as_index=False)[['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)']].sum()

In [217]:
data['Дата закупки'] = np.where(
    (data['Новая закупка (с 01.01.2023)'] > 0) & (data['Старая закупка (до 31.12.2022)'] > 0), 'Старые и новые',
    np.where(
        (data['Новая закупка (с 01.01.2023)'] > 0) & (data['Старая закупка (до 31.12.2022)'] == 0), 'Новые', 
        np.where(
            (data['Новая закупка (с 01.01.2023)'] == 0) & (data['Старая закупка (до 31.12.2022)'] > 0), 'Старые', 'Старые'
        )
    )
)

In [218]:
data = data.drop(columns=['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)'])

In [219]:
result = result.merge(data, how='left', on='ID Сайта')

In [220]:
result = result.drop(columns=['Дата закупки_x'])

In [221]:
result.columns = ['Код склада', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
    'ID Сайта', 'Товарное направление', 'Товарная группа',
    'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
    'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
    'Гендерный признак', 'Ценовая корзина', 'Группа цен',
    'Цена Розн., за шт', 'Чистый вес', 'Цена изделия на бирке',
    'Скидка на изделии', 'Запрет скидки',
    'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Опубликован',
    'Ссылка на изображение', 'Остаток', 'Количество продаж 14.03-19.06',
    'Сумма продаж 14.03-19.06', 'Чистый вес макс', 'Чистый вес медиана',
    'Чистый вес мин', 'Чистый вес размах, %', 'Цена изделия на бирке макс',
    'Цена изделия на бирке медиана', 'Цена изделия на бирке мин',
    'Базовая скидка макс', 'Базовая скидка мин', 'Цена со скидками макс',
    'Цена со скидками медиана', 'Цена со скидками мин',
    'Цена со скидками размах, %', 'Себестоимость макс',
    'Себестоимость медиана', 'Себестоимость мин', 'Себестоимость размах, %',
    'Наценка макс', 'Наценка медиана', 'Наценка мин', 'Запрет к размещению',
    'Цена со скидками Ecom', 'Сумма в себестоимости на остатках',
    'Цена за грамм макс', 'Цена за грамм медиана', 'Цена за грамм мин',
    'Количество с запрещенной скидкой', 'Количество с разрешенной скидкой',
    'Дата закупки']

In [222]:
result[['Чистый вес размах, %']]

,"Чистый вес размах, %"
0,0.111537
1,0.379001
2,0.163383
3,0.156837
4,0.095462
...,...
44602,-0.000000
44603,-0.000000
44604,-0.000000
44605,-0.000000


In [223]:
result['Размах по весу'] = np.where(
    result['Чистый вес размах, %'] >= 0.3, '3. Большой размах (>30%)',
    np.where(
        result['Чистый вес размах, %'] >= 0.1, '2. Средний размах (10-30%)', '1. Небольшой размах (<10%)'
    )
)

In [224]:
result['Размах по себестоимости'] = np.where(
    result['Себестоимость размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Себестоимость размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<10%)'
    )
)

In [225]:
result['Размах по цене со скидками'] = np.where(
    result['Цена со скидками размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Цена со скидками размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<10%)'
    )
)

In [226]:
result['Цена за грамм размах, %'] = (result['Цена за грамм макс'] / result['Цена за грамм мин'] - 1) * -1

In [227]:
result['Размах по цене за грамм'] = np.where(
    result['Цена за грамм размах, %'] >= 0.3, '3. Большой размах (>30%)',
    np.where(
        result['Цена за грамм размах, %'] >= 0.1, '2. Средний размах (10-30%)', '1. Небольшой размах (<10%)'
    )
)

In [228]:
result['Скидка разрешена'] = result['Количество с запрещенной скидкой'] + result['Количество с разрешенной скидкой']

In [229]:
result['Скидка разрешена'] = np.where(
    result['Скидка разрешена'].astype(str).str.contains('nan'), 'Да или нет', 'Да и нет'
)

In [230]:
result.columns

Index(['Код склада', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
       'ID Сайта', 'Товарное направление', 'Товарная группа',
       'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
       'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
       'Гендерный признак', 'Ценовая корзина', 'Группа цен',
       'Цена Розн., за шт', 'Чистый вес', 'Цена изделия на бирке',
       'Скидка на изделии', 'Запрет скидки',
       'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Опубликован',
       'Ссылка на изображение', 'Остаток', 'Количество продаж 14.03-19.06',
       'Сумма продаж 14.03-19.06', 'Чистый вес макс', 'Чистый вес медиана',
       'Чистый вес мин', 'Чистый вес размах, %', 'Цена изделия на бирке макс',
       'Цена изделия на бирке медиана', 'Цена изделия на бирке мин',
       'Базовая скидка макс', 'Базовая скидка мин', 'Цена со скидками макс',
       'Цена со скидками медиана', 'Цена со скидками мин',
       'Цена со скидками размах, %', 'Себестои

In [231]:
result['Цена изделия на бирке размах, %'] = (result['Цена изделия на бирке макс'] / result['Цена изделия на бирке мин'] - 1) * -1

In [232]:
result['Размах цена изделия на бирке'] = np.where(
    result['Цена изделия на бирке размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Цена изделия на бирке размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<20%)'
    )
)

In [233]:
result = result.drop(columns = ['Код склада', 'Чистый вес', 'Цена изделия на бирке', 'Скидка на изделии', 'Запрет скидки', 'Цена Закупки за 1 шт. (упр сс с НДС за единицу)',
                                'Запрет скидки'])

In [234]:
result = result[['Код товара', 'Артикул поставщика', 'Внутренний артикул', 'ID Сайта',
    'Товарное направление', 'Товарная группа', 'Товарный кластер', 'Проба',
    'Размер изделия', 'Тип изделия 1', 'Тип изделия 2', 'Тип изделия 3',
    'Дизайн', 'Бренд', 'Гендерный признак', 'Ценовая корзина', 'Группа цен',
    'Опубликован', 'Ссылка на изображение', 'Остаток', 'Цена Розн., за шт',
    'Количество продаж 14.03-19.06', 'Сумма продаж 14.03-19.06',
    'Чистый вес макс', 'Чистый вес медиана', 'Чистый вес мин',
    'Чистый вес размах, %', 'Цена изделия на бирке макс',
    'Цена изделия на бирке медиана', 'Цена изделия на бирке мин', 'Цена изделия на бирке размах, %',
    'Базовая скидка макс', 'Базовая скидка мин', 'Цена со скидками макс',
    'Цена со скидками медиана', 'Цена со скидками мин',
    'Цена со скидками размах, %', 'Себестоимость макс',
    'Себестоимость медиана', 'Себестоимость мин', 'Себестоимость размах, %',
    'Наценка макс', 'Наценка медиана', 'Наценка мин', 'Запрет к размещению',
    'Цена со скидками Ecom', 'Сумма в себестоимости на остатках',
    'Цена за грамм макс', 'Цена за грамм медиана', 'Цена за грамм мин', 'Цена за грамм размах, %',
    'Количество с запрещенной скидкой', 'Количество с разрешенной скидкой',
    'Размах по весу', 'Размах по себестоимости',
    'Размах по цене со скидками', 'Размах по цене за грамм', 'Размах цена изделия на бирке', 'Скидка разрешена', 'Дата закупки']]

In [240]:
tov = result.groupby(by = 'Товарная группа')['Остаток'].sum()

In [246]:
tov

Товарная группа
БК АКСЕССУАРЫ          98.0
БК КОЛЬЦА           68007.0
БК КОЛЬЦА ОБРУЧ    218880.0
БК ПЕЧАТКИ          18568.0
БК ПОДВЕС ДЕКОР     44555.0
БК ПОДВЕС КУЛЬТ    180520.0
БК СЕРЬГИ          217374.0
ДК КАМНИ                6.0
ДК КОЛЬЦА          239842.0
ДК КОЛЬЦА ОБРУЧ     33501.0
ДК ПЕЧАТКИ           6062.0
ДК ПОДВЕСКИ         65128.0
ДК СЕРЬГИ          167163.0
ИФ АКСЕССУАРЫ         433.0
ИФ КОЛЬЦА          242694.0
ИФ КОЛЬЦА ОБРУЧ     27511.0
ИФ ПЕЧАТКИ          26535.0
ИФ ПОДВЕС ДЕКОР    123752.0
ИФ ПОДВЕС КУЛЬТ     38167.0
ИФ СЕРЬГИ          331658.0
ОПТ БК               6164.0
ОПТ ИФ              12540.0
ОПТ КОНГО            1771.0
ОПТ ОБРУЧИ           4027.0
ОПТ ПДК              2938.0
ПДК КОЛЬЦА         186787.0
ПДК ПЕЧАТКИ         11563.0
ПДК ПОДВЕСКИ       102319.0
ПДК СЕРЬГИ         232844.0
ЦБ БРАСЛЕТЫ        228330.0
ЦБ ЦЕПИ            348324.0
Name: Остаток, dtype: float64

In [252]:
tov_sales = result.groupby(by = 'Товарная группа')['Количество продаж 14.03-19.06'].sum()

In [250]:
result['Доля % остатка ID отосительно остатка ТГ'] = np.nan
for i in result.index: 
    t = result['Товарная группа'][i]
    result['Доля % остатка ID отосительно остатка ТГ'] = result['Остаток'] / tov[t] 

In [267]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44607 entries, 0 to 44606
Data columns (total 62 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Код товара                                 44607 non-null  object 
 1   Артикул поставщика                         44607 non-null  object 
 2   Внутренний артикул                         44607 non-null  object 
 3   ID Сайта                                   44607 non-null  object 
 4   Товарное направление                       44607 non-null  object 
 5   Товарная группа                            44607 non-null  object 
 6   Товарный кластер                           44607 non-null  object 
 7   Проба                                      44607 non-null  object 
 8   Размер изделия                             44607 non-null  object 
 9   Тип изделия 1                              44607 non-null  object 
 10  Тип изделия 2         

In [268]:
result[result['Цена изделия на бирке макс', 'Цена изделия на бирке мин', 'Цена изделия на бирке медиана']]

KeyError: ('Цена изделия на бирке макс', 'Цена изделия на бирке мин', 'Цена изделия на бирке медиана')

In [265]:
result.columns

Index(['Код товара', 'Артикул поставщика', 'Внутренний артикул', 'ID Сайта',
       'Товарное направление', 'Товарная группа', 'Товарный кластер', 'Проба',
       'Размер изделия', 'Тип изделия 1', 'Тип изделия 2', 'Тип изделия 3',
       'Дизайн', 'Бренд', 'Гендерный признак', 'Ценовая корзина', 'Группа цен',
       'Опубликован', 'Ссылка на изображение', 'Остаток', 'Цена Розн., за шт',
       'Количество продаж 14.03-19.06', 'Сумма продаж 14.03-19.06',
       'Чистый вес макс', 'Чистый вес медиана', 'Чистый вес мин',
       'Чистый вес размах, %', 'Цена изделия на бирке макс',
       'Цена изделия на бирке медиана', 'Цена изделия на бирке мин',
       'Цена изделия на бирке размах, %', 'Базовая скидка макс',
       'Базовая скидка мин', 'Цена со скидками макс',
       'Цена со скидками медиана', 'Цена со скидками мин',
       'Цена со скидками размах, %', 'Себестоимость макс',
       'Себестоимость медиана', 'Себестоимость мин', 'Себестоимость размах, %',
       'Наценка макс', 'Наце

In [254]:
result['Доля % относительно продаж ТГ 14.03-19.06'] = np.nan
for i in result.index: 
    t = result['Товарная группа'][i]
    result['Доля % относительно продаж ТГ 14.03-19.06'] = result['Количество продаж 14.03-19.06'] / tov_sales[t] 

In [256]:

result.to_excel(r'C:\Users\Trenkin.Sergey\Desktop\notebooks\result.xlsx', index=False)

In [257]:
result.to_excel(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\Исследование цен\\в разрезе ID.xlsx', index=False)

In [258]:
result['Товарное направление'].value_counts()

Товарное направление
ПДК    15220
ДК     13820
ИФ      8328
БК      4796
ЦБ      2443
Name: count, dtype: int64

In [264]:
for i in result['Товарное направление'].unique():
    result[result['Товарное направление'] == i].to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\Исследование цен\\Исследование цен для {i} в разрезе ID.xlsx', index=False) 